In [66]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.3 MB/s 
     |████████████████████████████████| 2.2 MB 53.1 MB/s 


In [98]:
import pandas as pd
from Levenshtein import distance as levenshtein_distance

In [34]:
RAW_DATA_DIR = "../data/raw/numbeo/"
PROCESSED_DATA_DIR = "../data/processed/"
FINAL_DATA_DIR = "../data/final/"

In [35]:
df = pd.read_csv("numbeo_merged.csv")

In [36]:
df.head()

,Unnamed: 0,CountryName,CityName,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Cappuccino (regular),Coke/Pepsi (12 oz small bottle),Water (12 oz small bottle),"Milk (regular), (1 gallon)",...,Noise and Light Pollution,Water Pollution,Dissatisfaction to Spend Time in the City,Dissatisfaction with Green and Parks in the City,Quality of Life Index:,Traffic Index:,Time Index (in minutes):,Time Exp. Index:,Inefficiency Index:,CO2 Emission Index:
0,1,Albania,Tirana,5.02 $,33.48 $,5.44 $,1.30 $,1.29 $,0.66 $,4.80 $,...,58.89,72.60,69.27,67.99,90.80,122.19,38.53,1227.84,95.55,1509.06
1,2,Algeria,Algiers,2.84 $,21.32 $,4.62 $,0.90 $,0.49 $,0.19 $,3.30 $,...,39.86,58.57,56.41,72.30,87.36,247.82,52.71,8402.50,229.26,7796.43
2,4,Angola,Luanda,7.84 $,64.25 $,10.51 $,3.93 $,1.95 $,1.42 $,9.44 $,...,69.12,81.25,80.00,88.33,59.27,370.86,65.43,23369.50,426.01,17404.00
3,5,Argentina,Buenos Aires,6.56 $,25.00 $,5.00 $,1.77 $,1.00 $,0.80 $,2.91 $,...,55.26,54.85,36.22,34.05,100.81,204.58,49.19,5821.10,214.35,4153.19
4,6,Argentina,Cordoba,5.00 $,21.00 $,5.74 $,1.50 $,1.02 $,0.90 $,3.00 $,...,63.33,58.33,56.67,66.67,120.88,138.18,33.00,318.01,122.77,5816.29


In [37]:
for c in df.columns:
    print(f"'{c}',")

'Unnamed: 0',
'CountryName',
'CityName',
'Meal, Inexpensive Restaurant',
'Meal for 2 People, Mid-range Restaurant, Three-course',
'McMeal at McDonalds (or Equivalent Combo Meal)',
'Cappuccino (regular)',
'Coke/Pepsi (12 oz small bottle)',
'Water (12 oz small bottle)',
'Milk (regular), (1 gallon)',
'Loaf of Fresh White Bread (1 lb)',
'Rice (white), (1 lb)',
'Eggs (regular) (12)',
'Local Cheese (1 lb)',
'Chicken Fillets (1 lb)',
'Apples (1 lb)',
'Banana (1 lb)',
'Oranges (1 lb)',
'Tomato (1 lb)',
'Potato (1 lb)',
'Onion (1 lb)',
'Lettuce (1 head)',
'Water (1.5 liter bottle)',
'Cigarettes 20 Pack (Marlboro)',
'One-way Ticket (Local Transport)',
'Monthly Pass (Regular Price)',
'Taxi Start (Normal Tariff)',
'Taxi 1 mile (Normal Tariff)',
'Taxi 1hour Waiting (Normal Tariff)',
'Gasoline (1 gallon)',
'Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car)',
'Toyota Corolla Sedan 1.6l 97kW Comfort (Or Equivalent New Car)',
'Basic (Electricity, Heating, Cooling, Water, Garbage) for 915 sq f

In [38]:
signif_cols = [
    'CountryName',
    'CityName',
    'Meal, Inexpensive Restaurant',
    'Meal for 2 People, Mid-range Restaurant, Three-course',
    'McMeal at McDonalds (or Equivalent Combo Meal)',
    'Cappuccino (regular)',
    'Cigarettes 20 Pack (Marlboro)',
    'One-way Ticket (Local Transport)',
    'Monthly Pass (Regular Price)',
    'Taxi Start (Normal Tariff)',
    'Taxi 1 mile (Normal Tariff)',
    'Gasoline (1 gallon)',
    'Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car)',
    'Basic (Electricity, Heating, Cooling, Water, Garbage) for 915 sq ft Apartment',
    'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)',
    'Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child',
    'International Primary School, Yearly for 1 Child',
    'Apartment (1 bedroom) in City Centre',
    'Apartment (1 bedroom) Outside of Centre',
    'Apartment (3 bedrooms) in City Centre',
    'Apartment (3 bedrooms) Outside of Centre',
    'Price per Square Feet to Buy Apartment in City Centre',
    'Price per Square Feet to Buy Apartment Outside of Centre',
    'Average Monthly Net Salary (After Tax)',
    'Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate',
    'Level of crime',
    'Skill and competency of medical staff',
    'Air Pollution',
    'Drinking Water Pollution and Inaccessibility',
    'Dissatisfaction with Green and Parks in the City',
    'Quality of Life Index:',
    'Traffic Index:'
]

In [39]:
len(signif_cols)

32

In [40]:
df1 = df[signif_cols]

In [41]:
df1['Meal for 2 People, Mid-range Restaurant, Three-course'] = df1['Meal for 2 People, Mid-range Restaurant, Three-course'].replace('[^-.0-9]', '', regex=True).astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
df1.to_csv("numbeo_final.csv")

In [43]:
df1 = pd.read_csv("numbeo_final.csv")

In [44]:
df1.head()

,Unnamed: 0,CountryName,CityName,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Cappuccino (regular),Cigarettes 20 Pack (Marlboro),One-way Ticket (Local Transport),Monthly Pass (Regular Price),...,Price per Square Feet to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",Level of crime,Skill and competency of medical staff,Air Pollution,Drinking Water Pollution and Inaccessibility,Dissatisfaction with Green and Parks in the City,Quality of Life Index:,Traffic Index:
0,0,Albania,Tirana,5.02 $,33.48,5.44 $,1.30 $,2.93 $,0.33 $,13.39 $,...,89.40 $,431.63 $,4.47,45.57,52.31,80.73,73.82,67.99,90.80,122.19
1,1,Algeria,Algiers,2.84 $,21.32,4.62 $,0.90 $,2.27 $,0.28 $,10.66 $,...,96.97 $,275.22 $,7.19,55.24,53.06,60.00,49.36,72.30,87.36,247.82
2,2,Angola,Luanda,7.84 $,64.25,10.51 $,3.93 $,2.00 $,0.59 $,31.78 $,...,320.40 $,"1,428.77 $",25.00,76.73,34.62,85.00,78.12,88.33,59.27,370.86
3,3,Argentina,Buenos Aires,6.56 $,25.00,5.00 $,1.77 $,2.00 $,0.16 $,10.00 $,...,192.85 $,404.58 $,44.23,74.82,81.46,45.00,30.57,34.05,100.81,204.58
4,4,Argentina,Cordoba,5.00 $,21.00,5.74 $,1.50 $,2.45 $,0.65 $,26.09 $,...,74.32 $,522.22 $,50.00,73.95,84.82,65.00,45.00,66.67,120.88,138.18


In [45]:
df1['id'] = df1['CountryName'] + df1['CityName']

In [46]:
df1['id']

0                AlbaniaTirana
1               AlgeriaAlgiers
2                 AngolaLuanda
3        ArgentinaBuenos Aires
4             ArgentinaCordoba
                ...           
759           VenezuelaCaracas
760             VietnamDa Nang
761               VietnamHanoi
762    VietnamHo Chi Minh City
763             ZimbabweHarare
Name: id, Length: 764, dtype: object

In [47]:
df1['id'].nunique()

764

In [48]:
df_u = df1.drop_duplicates(['id'])

In [49]:
df_u

,Unnamed: 0,CountryName,CityName,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Cappuccino (regular),Cigarettes 20 Pack (Marlboro),One-way Ticket (Local Transport),Monthly Pass (Regular Price),...,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",Level of crime,Skill and competency of medical staff,Air Pollution,Drinking Water Pollution and Inaccessibility,Dissatisfaction with Green and Parks in the City,Quality of Life Index:,Traffic Index:,id
0,0,Albania,Tirana,5.02 $,33.48,5.44 $,1.30 $,2.93 $,0.33 $,13.39 $,...,431.63 $,4.47,45.57,52.31,80.73,73.82,67.99,90.80,122.19,AlbaniaTirana
1,1,Algeria,Algiers,2.84 $,21.32,4.62 $,0.90 $,2.27 $,0.28 $,10.66 $,...,275.22 $,7.19,55.24,53.06,60.00,49.36,72.30,87.36,247.82,AlgeriaAlgiers
2,2,Angola,Luanda,7.84 $,64.25,10.51 $,3.93 $,2.00 $,0.59 $,31.78 $,...,"1,428.77 $",25.00,76.73,34.62,85.00,78.12,88.33,59.27,370.86,AngolaLuanda
3,3,Argentina,Buenos Aires,6.56 $,25.00,5.00 $,1.77 $,2.00 $,0.16 $,10.00 $,...,404.58 $,44.23,74.82,81.46,45.00,30.57,34.05,100.81,204.58,ArgentinaBuenos Aires
4,4,Argentina,Cordoba,5.00 $,21.00,5.74 $,1.50 $,2.45 $,0.65 $,26.09 $,...,522.22 $,50.00,73.95,84.82,65.00,45.00,66.67,120.88,138.18,ArgentinaCordoba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,759,Venezuela,Caracas,10.00 $,50.00,10.00 $,1.65 $,3.00 $,0.35 $,30.00 $,...,191.67 $,27.00,94.12,47.38,56.72,69.23,64.52,65.75,150.20,VenezuelaCaracas
760,760,Vietnam,Da Nang,1.47 $,16.75,6.28 $,1.28 $,1.23 $,0.29 $,6.28 $,...,303.59 $,7.50,18.96,80.00,50.00,47.50,38.10,128.91,90.28,VietnamDa Nang
761,761,Vietnam,Hanoi,2.09 $,18.84,5.40 $,1.96 $,1.47 $,0.29 $,8.37 $,...,473.42 $,9.86,32.66,57.56,84.52,57.72,57.14,98.33,106.78,VietnamHanoi
762,762,Vietnam,Ho Chi Minh City,2.09 $,24.82,4.19 $,2.09 $,1.26 $,0.29 $,5.65 $,...,501.66 $,9.18,54.33,60.55,87.57,62.15,66.48,66.53,117.93,VietnamHo Chi Minh City


In [50]:
float_cols = [
    'Meal, Inexpensive Restaurant',
    'Meal for 2 People, Mid-range Restaurant, Three-course',
    'McMeal at McDonalds (or Equivalent Combo Meal)',
    'Cappuccino (regular)',
    'Cigarettes 20 Pack (Marlboro)',
    'One-way Ticket (Local Transport)',
    'Monthly Pass (Regular Price)',
    'Taxi Start (Normal Tariff)',
    'Taxi 1 mile (Normal Tariff)',
    'Gasoline (1 gallon)',
    'Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car)',
    'Basic (Electricity, Heating, Cooling, Water, Garbage) for 915 sq ft Apartment',
    'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)',
    'Preschool (or Kindergarten), Full Day, Private, Monthly for 1 Child',
    'International Primary School, Yearly for 1 Child',
    'Apartment (1 bedroom) in City Centre',
    'Apartment (1 bedroom) Outside of Centre',
    'Apartment (3 bedrooms) in City Centre',
    'Apartment (3 bedrooms) Outside of Centre',
    'Price per Square Feet to Buy Apartment in City Centre',
    'Price per Square Feet to Buy Apartment Outside of Centre',
    'Average Monthly Net Salary (After Tax)',
    'Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate',
    'Level of crime',
    'Skill and competency of medical staff',
    'Air Pollution',
    'Drinking Water Pollution and Inaccessibility',
    'Dissatisfaction with Green and Parks in the City',
    'Quality of Life Index:',
    'Traffic Index:'
]

In [51]:
for c in float_cols:
    df_u[c] = df_u[c].replace('[^-.0-9]', '', regex=True).astype(float)

In [52]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 764 entries, 0 to 763
Data columns (total 34 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   Unnamed: 0                                                                     764 non-null    int64  
 1   CountryName                                                                    764 non-null    object 
 2   CityName                                                                       764 non-null    object 
 3   Meal, Inexpensive Restaurant                                                   764 non-null    float64
 4   Meal for 2 People, Mid-range Restaurant, Three-course                          764 non-null    float64
 5   McMeal at McDonalds (or Equivalent Combo Meal)                                 764 non-null    float64
 6   Cappuccino (regular)      

In [53]:
df_u.to_csv("numbeo.csv")  

In [54]:
df_u = df_u.drop(['Unnamed: 0'],axis=1)

In [55]:
df_u = df_u.dropna()

In [56]:
df_u = df_u.reset_index(drop=True)

In [57]:
df_u

,CountryName,CityName,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Cappuccino (regular),Cigarettes 20 Pack (Marlboro),One-way Ticket (Local Transport),Monthly Pass (Regular Price),Taxi Start (Normal Tariff),...,Average Monthly Net Salary (After Tax),"Mortgage Interest Rate in Percentages (%), Yearly, for 20 Years Fixed-Rate",Level of crime,Skill and competency of medical staff,Air Pollution,Drinking Water Pollution and Inaccessibility,Dissatisfaction with Green and Parks in the City,Quality of Life Index:,Traffic Index:,id
0,Albania,Tirana,5.02,33.48,5.44,1.30,2.93,0.33,13.39,2.51,...,431.63,4.47,45.57,52.31,80.73,73.82,67.99,90.80,122.19,AlbaniaTirana
1,Algeria,Algiers,2.84,21.32,4.62,0.90,2.27,0.28,10.66,0.36,...,275.22,7.19,55.24,53.06,60.00,49.36,72.30,87.36,247.82,AlgeriaAlgiers
2,Angola,Luanda,7.84,64.25,10.51,3.93,2.00,0.59,31.78,1.67,...,1428.77,25.00,76.73,34.62,85.00,78.12,88.33,59.27,370.86,AngolaLuanda
3,Argentina,Buenos Aires,6.56,25.00,5.00,1.77,2.00,0.16,10.00,1.00,...,404.58,44.23,74.82,81.46,45.00,30.57,34.05,100.81,204.58,ArgentinaBuenos Aires
4,Argentina,Cordoba,5.00,21.00,5.74,1.50,2.45,0.65,26.09,1.11,...,522.22,50.00,73.95,84.82,65.00,45.00,66.67,120.88,138.18,ArgentinaCordoba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,Venezuela,Caracas,10.00,50.00,10.00,1.65,3.00,0.35,30.00,4.00,...,191.67,27.00,94.12,47.38,56.72,69.23,64.52,65.75,150.20,VenezuelaCaracas
760,Vietnam,Da Nang,1.47,16.75,6.28,1.28,1.23,0.29,6.28,0.54,...,303.59,7.50,18.96,80.00,50.00,47.50,38.10,128.91,90.28,VietnamDa Nang
761,Vietnam,Hanoi,2.09,18.84,5.40,1.96,1.47,0.29,8.37,0.67,...,473.42,9.86,32.66,57.56,84.52,57.72,57.14,98.33,106.78,VietnamHanoi
762,Vietnam,Ho Chi Minh City,2.09,24.82,4.19,2.09,1.26,0.29,5.65,0.50,...,501.66,9.18,54.33,60.55,87.57,62.15,66.48,66.53,117.93,VietnamHo Chi Minh City


In [58]:
df_u.to_csv("numbeo.csv", index=False)

In [59]:
df_u.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 33 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   CountryName                                                                    764 non-null    object 
 1   CityName                                                                       764 non-null    object 
 2   Meal, Inexpensive Restaurant                                                   764 non-null    float64
 3   Meal for 2 People, Mid-range Restaurant, Three-course                          764 non-null    float64
 4   McMeal at McDonalds (or Equivalent Combo Meal)                                 764 non-null    float64
 5   Cappuccino (regular)                                                           764 non-null    float64
 6   Cigarettes 20 Pack (Marlbo

Wiki merge

In [60]:
wiki_sun = pd.read_csv("wiki_sunshine.csv")
wiki_temp = pd.read_csv("wiki_temperature.csv")

In [77]:
#id_list1 = df_u['id']

wiki_sun['id'] = wiki_sun['Country'] + wiki_sun['City']

wiki_temp['id'] = wiki_temp['Country'] + wiki_temp['City']

In [74]:
id_list2

0            Ivory CoastGagnoa
1            Ivory CoastBouaké
2           Ivory CoastAbidjan
3           Ivory CoastOdienné
4             Ivory CoastFerké
                ...           
385    New ZealandChristchurch
386      New ZealandWellington
387        New ZealandAuckland
388                   FijiSuva
389         New ZealandDunedin
Length: 390, dtype: object

In [93]:
list_as_set = set(df_u['id'])
intersection = list_as_set.intersection(wiki_temp['id'])
len(intersection)

198

In [114]:
for i in range(len(wiki_temp['id'])):
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('á', 'a')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('ó', 'o')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('é', 'e')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('í', 'i')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('â', 'a')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('ã', 'a')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('ł', 'l')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('ü', 'u')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('İ', 'I')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('ș', 's')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('ă', 'a')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace(' ', '')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].replace('-', '')
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].lower()
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].split(',')[0]
    wiki_temp['id'].iloc[i] = wiki_temp['id'].iloc[i].split('(')[0]
    if wiki_temp['id'].iloc[i] == 'polandcracow':
        wiki_temp['id'].iloc[i] = 'polandkrakow'

    if wiki_temp['id'].iloc[i] == 'denmarkarhus':
        wiki_temp['id'].iloc[i] = 'denmarkaarhus'

    if wiki_temp['id'].iloc[i] == 'ethiopiaaddisabeba':
        wiki_temp['id'].iloc[i] = 'ethiopiaaddisababa'
    

for i in range(len(wiki_sun['id'])):
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('á', 'a')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('ó', 'o')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('é', 'e')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('í', 'i')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('â', 'a')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('ã', 'a')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('ł', 'l')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('ü', 'u')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('İ', 'I')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('ș', 's')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('ă', 'a')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace(' ', '')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].replace('-', '')
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].lower()
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].split(',')[0]
    wiki_sun['id'].iloc[i] = wiki_sun['id'].iloc[i].split('(')[0]
    if wiki_sun['id'].iloc[i] == 'polandcracow':
        wiki_sun['id'].iloc[i] = 'polandkrakow'
    
    if wiki_sun['id'].iloc[i] == 'denmarkarhus':
        wiki_sun['id'].iloc[i] = 'denmarkaarhus'

    if wiki_sun['id'].iloc[i] == 'ethiopiaaddisabeba':
        wiki_sun['id'].iloc[i] = 'ethiopiaaddisababa'


for i in range(len(df_u['id'])):
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('á', 'a')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('ó', 'o')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('é', 'e')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('í', 'i')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('â', 'a')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('ã', 'a')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('ł', 'l')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('ü', 'u')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('İ', 'I')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('ș', 's')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('ă', 'a')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace(' ', '')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].replace('-', '')
    df_u['id'].iloc[i] = df_u['id'].iloc[i].lower()
    df_u['id'].iloc[i] = df_u['id'].iloc[i].split(',')[0]
    df_u['id'].iloc[i] = df_u['id'].iloc[i].split('(')[0]
    if df_u['id'].iloc[i] == 'polandcracow':
        df_u['id'].iloc[i] = 'polandkrakow'
        
    if df_u['id'].iloc[i] == 'denmarkarhus':
        df_u['id'].iloc[i] = 'denmarkaarhus'

    if df_u['id'].iloc[i] == 'ethiopiaaddisabeba':
        df_u['id'].iloc[i] = 'ethiopiaaddisababa'

In [121]:
wiki_temp.head()

,Unnamed: 0,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year,id
0,0,Algeria,Algiers,11.2,11.9,12.8,14.7,17.7,21.3,24.6,25.2,23.2,19.4,15.2,12.1,17.4,algeriaalgiers
1,1,Algeria,Tamanrasset,12.8,15.0,18.1,22.2,26.1,28.9,28.7,28.2,26.5,22.4,17.3,13.9,21.7,algeriatamanrasset
2,2,Algeria,Reggane,16.0,18.2,23.1,27.9,32.2,36.4,39.8,38.4,35.5,29.2,22.0,17.8,28.3,algeriareggane
3,3,Angola,Luanda,26.7,28.5,28.6,28.2,27.0,23.9,22.1,22.1,23.5,25.2,26.7,26.9,25.8,angolaluanda
4,4,Benin,Cotonou,27.3,28.5,28.9,28.6,27.8,26.5,25.8,25.6,26.0,26.7,27.6,27.3,27.2,benincotonou


In [122]:
wiki_sun = wiki_sun.drop(wiki_sun.iloc[:, 0:3],axis = 1)
wiki_temp = wiki_temp.drop(wiki_temp.iloc[:, 0:3],axis = 1)



In [125]:
wiki_sun.columns = ['sun_' + x for x in wiki_sun.columns]
wiki_temp.columns = ['temp_' + x for x in wiki_temp.columns]

In [129]:
wiki_sun = wiki_sun.rename(columns={'sun_id': 'id'})
wiki_temp = wiki_temp.rename(columns={'temp_id': 'id'})

In [132]:
df_u = pd.merge(df_u, wiki_sun, how="left", on=["id"])
df_u = pd.merge(df_u, wiki_temp, how="left", on=["id"])

In [133]:
df_u

,CountryName,CityName,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Cappuccino (regular),Cigarettes 20 Pack (Marlboro),One-way Ticket (Local Transport),Monthly Pass (Regular Price),Taxi Start (Normal Tariff),...,temp_Apr,temp_May,temp_Jun,temp_Jul,temp_Aug,temp_Sep,temp_Oct,temp_Nov,temp_Dec,temp_Year
0,Albania,Tirana,5.02,33.48,5.44,1.30,2.93,0.33,13.39,2.51,...,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1,15.2
1,Algeria,Algiers,2.84,21.32,4.62,0.90,2.27,0.28,10.66,0.36,...,14.7,17.7,21.3,24.6,25.2,23.2,19.4,15.2,12.1,17.4
2,Angola,Luanda,7.84,64.25,10.51,3.93,2.00,0.59,31.78,1.67,...,28.2,27.0,23.9,22.1,22.1,23.5,25.2,26.7,26.9,25.8
3,Argentina,Buenos Aires,6.56,25.00,5.00,1.77,2.00,0.16,10.00,1.00,...,17.9,14.5,11.7,11.0,12.8,14.6,17.9,20.7,23.3,17.9
4,Argentina,Cordoba,5.00,21.00,5.74,1.50,2.45,0.65,26.09,1.11,...,18.0,14.5,11.4,10.8,13.3,15.7,19.5,21.8,23.7,18.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,Venezuela,Caracas,10.00,50.00,10.00,1.65,3.00,0.35,30.00,4.00,...,22.7,23.2,22.6,22.2,22.3,22.7,22.4,22.0,20.7,22.0
760,Vietnam,Da Nang,1.47,16.75,6.28,1.28,1.23,0.29,6.28,0.54,...,26.4,28.3,29.2,29.3,29.0,27.5,25.9,24.1,22.1,25.8
761,Vietnam,Hanoi,2.09,18.84,5.40,1.96,1.47,0.29,8.37,0.67,...,23.9,27.4,28.9,29.2,28.6,27.5,24.9,21.5,18.2,23.6
762,Vietnam,Ho Chi Minh City,2.09,24.82,4.19,2.09,1.26,0.29,5.65,0.50,...,29.2,28.8,27.8,27.5,27.4,27.2,27.0,26.7,26.0,27.4


In [134]:
df_u.to_csv('numbeo+wiki')